<a href="https://colab.research.google.com/github/PaulToronto/NYC---Taxi-and-Limousine-Commission/blob/main/Route_Optimization_with_NYC_TLC_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Route Optimization with NYC TLC Data

## Imports

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import gdown

## Functions

In [2]:
get_data_url = lambda id: rf'https://drive.google.com/uc?id={id}'
gdown_download = lambda id, file_name: gdown.download(get_data_url(id), file_name, quiet=False)

def euclidean_distance_miles(pickup_lon, dropoff_lon, pickup_lat, dropoff_lat):
    distance_feet = np.sqrt((pickup_lon - dropoff_lon)**2 + (pickup_lat - dropoff_lat)**2)
    distance_miles = distance_feet / 5280
    return distance_miles

## High Volume For-Hire Vehicle Trips and Taxi Zones Datasets

- Includes Uber and Lyft trips

### Trips

In [3]:
id = '1rEgVTd1wkF7R5jScMqwAIaX9Fm4w-JzO'
file_name = 'fhv_hv_july_2024.parquet'
gdown_download(id, file_name)

Downloading...
From (original): https://drive.google.com/uc?id=1rEgVTd1wkF7R5jScMqwAIaX9Fm4w-JzO
From (redirected): https://drive.google.com/uc?id=1rEgVTd1wkF7R5jScMqwAIaX9Fm4w-JzO&confirm=t&uuid=5bfed70f-7144-4bd8-b4d1-f10653ae50f3
To: /content/fhv_hv_july_2024.parquet
100%|██████████| 468M/468M [00:02<00:00, 167MB/s]


'fhv_hv_july_2024.parquet'

In [4]:
trips = pd.read_parquet(file_name)

In [5]:
trips['pickup_hour'] = trips['pickup_datetime'].dt.hour
trips['dropoff_hour'] = trips['dropoff_datetime'].dt.hour

### Zones

In [6]:
id='1fGdhMuUlq3ZdGlBt8I4w5kIeXiovIBzS'
file_name = 'taxi_zones.zip'
gdown_download(id, file_name)

Downloading...
From: https://drive.google.com/uc?id=1fGdhMuUlq3ZdGlBt8I4w5kIeXiovIBzS
To: /content/taxi_zones.zip
100%|██████████| 1.03M/1.03M [00:00<00:00, 180MB/s]


'taxi_zones.zip'

In [7]:
# zone has multiple rows for
#. - Governor's Island/Ellis Island/Liberty Island
#. - Corona
# I'll figure out why later
zones = gpd.read_file(file_name)

In [8]:
# create a centroid lon and lat
zones['centroid_lon'] = zones['geometry'].centroid.x
zones['centroid_lat'] = zones['geometry'].centroid.y

zones.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   OBJECTID      263 non-null    int32   
 1   Shape_Leng    263 non-null    float64 
 2   Shape_Area    263 non-null    float64 
 3   zone          263 non-null    object  
 4   LocationID    263 non-null    int32   
 5   borough       263 non-null    object  
 6   geometry      263 non-null    geometry
 7   centroid_lon  263 non-null    float64 
 8   centroid_lat  263 non-null    float64 
dtypes: float64(4), geometry(1), int32(2), object(2)
memory usage: 16.6+ KB


In [9]:
zones[zones.LocationID.duplicated()]

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,centroid_lon,centroid_lat
56,57,0.019271,0.000018,Corona,56,Queens,"POLYGON ((1025447.751 212499.788, 1024585.351 ...",1.024817e+06,213218.634924
103,104,0.021221,0.000012,Governor's Island/Ellis Island/Liberty Island,103,Manhattan,"POLYGON ((973172.666 194632.348, 973310.63 194...",9.729448e+05,193859.360777
104,105,0.077425,0.000369,Governor's Island/Ellis Island/Liberty Island,103,Manhattan,"POLYGON ((979605.759 191880.575, 979978.435 19...",9.789606e+05,190219.673877


## Joined Data

In [10]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19182934 entries, 0 to 19182933
Data columns (total 26 columns):
 #   Column                Dtype         
---  ------                -----         
 0   hvfhs_license_num     object        
 1   dispatching_base_num  object        
 2   originating_base_num  object        
 3   request_datetime      datetime64[us]
 4   on_scene_datetime     datetime64[us]
 5   pickup_datetime       datetime64[us]
 6   dropoff_datetime      datetime64[us]
 7   PULocationID          int32         
 8   DOLocationID          int32         
 9   trip_miles            float64       
 10  trip_time             int64         
 11  base_passenger_fare   float64       
 12  tolls                 float64       
 13  bcf                   float64       
 14  sales_tax             float64       
 15  congestion_surcharge  float64       
 16  airport_fee           float64       
 17  tips                  float64       
 18  driver_pay            float64       
 19

In [11]:
zones_PU = zones.add_suffix('_PU').rename(columns={'LocationID_PU': 'PULocationID'})
zones_PU = gpd.GeoDataFrame(zones_PU, geometry='geometry_PU', crs=zones.crs)

zones_DO = zones.add_suffix('_DO').rename(columns={'LocationID_DO': 'DOLocationID'})
zones_DO = gpd.GeoDataFrame(zones_DO, geometry='geometry_DO', crs=zones.crs)

del zones

trips = trips.merge(
    zones_PU, on='PULocationID', how='inner'
).merge(
    zones_DO, on='DOLocationID', how='inner'
)

trips = gpd.GeoDataFrame(trips, geometry='geometry_PU', crs=zones_PU.crs)

In [12]:
trips.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18475615 entries, 0 to 18475614
Data columns (total 42 columns):
 #   Column                Dtype         
---  ------                -----         
 0   hvfhs_license_num     object        
 1   dispatching_base_num  object        
 2   originating_base_num  object        
 3   request_datetime      datetime64[us]
 4   on_scene_datetime     datetime64[us]
 5   pickup_datetime       datetime64[us]
 6   dropoff_datetime      datetime64[us]
 7   PULocationID          int32         
 8   DOLocationID          int32         
 9   trip_miles            float64       
 10  trip_time             int64         
 11  base_passenger_fare   float64       
 12  tolls                 float64       
 13  bcf                   float64       
 14  sales_tax             float64       
 15  congestion_surcharge  float64       
 16  airport_fee           float64       
 17  tips                  float64       
 18  driver_pay            float64   

In [13]:
# use trips.set_geometry() to set
#. geometry to `geometry_DO` as needed
trips.active_geometry_name

'geometry_PU'

### Attributes of Joined Data

1. **`hvfhs_license_num`**: The TLC license number of the HVFHS base or business. As of September 2019, the HVFHS licensees are the following:
 - HV0002: Juno
 - HV0003: Uber
 - HV0004: Via
 - HV0005: Lyft
2. **`dispatching_base_num`**: The TLC Base License Number of the base that dispatched the trip
3. **`originating_base_num`**: Base number of the base that received the original trip request
4. **`request_datetime`**: Date/time when passenger requested to be picked up
5. **`on_scene_datetime`**: Date/time when driver arrived at the pick-up location (Accessible
Vehicles-only)
6. **`pickup_datetime`**: The date and time of the trip pick-up
7. **`dropoff_datetime`**: The date and time of the trip drop-off
8. **`PULocationID`**: The date and time of the trip pick-up
9. **`DOLocationID`**: The date and time of the trip drop-off
10. **`trip_miles`**: Total miles for passenger trip
11. **`trip_time`**: Total time in seconds for passenger trip
12. **`base_passenger_fare`**: Base passenger fare before tolls, tips, taxes, and fees
13. **`tolls`**: Total amount of all tolls paid in trip
14. **`sales_tax`**: Total amount collected in trip for NYS sales tax
15. **`congestion_surcharge`**: Total amount collected in trip for NYS congestion surcharge
16. **`airport_fee`**: $2.50 for both drop off and pick up at LaGuardia, Newark, and John F. Kennedy airports
17. **`tips`**: Total amount of tips received from passenger
18. **`driver_pay`**: Total driver pay (not including tolls or tips and net of commission surcharges, or taxes)
19. **`shared_request_flag`**: Did the passenger agree to a shared/pooled ride, regardless of whether they were matched? (Y/N)
20. **`shared_match_flag`**: Did the passenger share the vehicle with another passenger who booked separately at any point during the trip? (Y/N)
21. **`access_a_ride_flag`**: Was the trip administered on behalf of the Metropolitan Transportation Authority (MTA)? (Y/N)
22. **`wav_request_flag`**: Did the passenger request a wheelchair-accessible vehicle (WAV)? (Y/N)
23. **`wav_match_flag`**: Did the trip occur in a wheelchair-accessible vehicle (WAV)? (Y/N)
24. **`pickup_hour`, `dropoff_hour`**: Created from `pickup_datetime` and `dropoff_datetime`
25. **`Shape_Leng_PU`, `Shape_Leng_DO'`**: Length of the geometry
26. **`Shape_Area_PU`, `Shape_Area_DO`**: Area of the geometry
27. **`zone_PU`, `zone_DO`**: Name of zone
28. **`borough_PU`, `borough_DO`**: Name of borough
29. **`geometry_PU`, `geometry_DO`**: geometry
30. **`centroid_lon_PU`, `centroid_lon_DO`**: Longitude of the centroid
31. **`centroid_lat_PU`, `centroid_lat_DO`**: Latitude of the centroid

## Features needed for a Route Optimization Model

For now, I am focusing on Rides data, but I will investigate other data that is applicable to a route optimization model when time permits:

- Traffic data
- Events data
- Weather data
- Road network data
- Fuel and energy consumption data
- User or driver data



### Ride data attributes (target variables covered later in this notebook)

1. Pickup and Drop-of Coordinates
    - We have:
        - `centroid_lon_PU`, `centroid_lat_PU`
        - `centroid_lon_DO`, `centroid_lat_DO`
    - The trouble is that these are not exact locations, but the centroids of large zones which will impact the accuracy of the model. There are ways to mitigate this problem:
        1. Zone Adjacency: makes use of distance between centroids
        2. Distance-Based Weighting: makes use of distance between centroids
2. Latitude and Longitude for positions along the route
    - We don't have this
    - Ways to mitigate this problem:
        - Use shortest path between start and end points
            - ```python
import osmnx as ox
import networkx as nx
# Download the road network for New York City
G = ox.graph_from_place("New York City, New York, USA", network_type='drive')
# Define the coordinates for pickup and dropoff zone centroids
pickup_coords = (40.730610, -73.935242)  # Example: Pickup at some NYC location
dropoff_coords = (40.7128, -74.0060)  # Example: Dropoff in Manhattan
# Find the nearest nodes in the road network
pickup_node = ox.distance.nearest_nodes(G, X=pickup_coords[1], Y=pickup_coords[0])
dropoff_node = ox.distance.nearest_nodes(G, X=dropoff_coords[1], Y=dropoff_coords[0])
# Calculate the shortest path between the two nodes
route = nx.shortest_path(G, pickup_node, dropoff_node, weight='length')
# Plot the estimated route on the map
ox.plot_graph_route(G, route, route_linewidth=2, node_size=0)
```
        - Incorporate Historical Traffic Patterns Between Zones
            - This provides a baseline travel time estimate for specific zone pairs
            - allows you to predict delays during peak hours or congestd
            - ```python
# Aggregate data to calculate average trip times by zone pair and time window
taxi_data['pickup_hour'] = taxi_data['pickup_datetime'].dt.hour
grouped = taxi_data.groupby(['PULocationID', 'DOLocationID', 'pickup_hour']).agg(
    avg_trip_time=('trip_duration', 'mean')
).reset_index()
print(grouped.head())
```
        - Use External Traffic Data to Model In-Ride Dynamics
            - ```python
import googlemaps
# Initialize the Google Maps API client
gmaps = googlemaps.Client(key='YOUR_GOOGLE_MAPS_API_KEY')
# Request travel time between two points
directions = gmaps.directions("40.730610,-73.935242", "40.7128,-74.0060", departure_time='now')
# Extract the travel time from the response (in seconds)
travel_time = directions[0]['legs'][0]['duration']['value'] / 60  # Convert to minutes
print(f"Estimated Travel Time: {travel_time} minutes")
```
        - Use Graph-Based Models for Zone Pair Dynamics
            - ```python
import networkx as nx
# Create a graph with zones as nodes and average travel time as edge weights
G = nx.Graph()
# Add nodes for each zone
zones = taxi_data[['PULocationID', 'DOLocationID']].stack().unique()
G.add_nodes_from(zones)
# Add edges with average travel time between zones
for _, row in grouped.iterrows():
    G.add_edge(row['PULocationID'], row['DOLocationID'], weight=row['avg_trip_time'])
# Find the shortest path (in terms of travel time) between two zones
shortest_path = nx.shortest_path(G, source=1, target=2, weight='weight')
print(f"Shortest path based on travel time: {shortest_path}")

            ```
3. Time stamps for when GPS data is captured
    - We have this, but only for start and end points
4. Number of Stoplights or Intersections
    - We can probably find this

1. Pickup and Drop-off Coordinates:
    



2. Latitude and Longitude for postions along the route

- We don't have this

3. Time stamps for when GPS data is captured
    - We only have this for the pickup and drop-off times

#### Euclidean Distance:

Used for:

1. Zone Adjacency
2. Distance Based Weighting

- Note that the dataset already has `trip_miles` field. I don't know exactly how this was obtained, but it is not exactly equal to the euclidean distance, which I have called `trip_miles_calculated`
- For our purpose, I believe it is best to use `trip_miles_calculated`
    - `trip_miles` is mismathed with the centroids
    - `trip_miles` may be a better predictor for NYC, but our ultimate goal is not a model with predictive power in NYC
    - `trip_miles_calculated` is more generalizable and transferable to other cities
    - Ultimately, an effort should be made to collect exact pickup and dropoff locations in the future, in which case, this decision no longer matters

In [14]:
trips['trip_miles_calculated'] = euclidean_distance_miles(trips['centroid_lon_PU'],
                                                          trips['centroid_lon_DO'],
                                                          trips['centroid_lat_PU'],
                                                          trips['centroid_lat_DO'])

In [15]:
trips['trip_miles_calculated'].head()

,trip_miles_calculated
0,4.540892
1,2.825206
2,3.075012
3,7.102568
4,0.950297


In [16]:
trips['trip_miles'].head()

,trip_miles
0,8.84
1,3.96
2,5.61
3,11.24
4,1.60


## Target

Given Use Case:

AI-driven route optimization for drivers to minimize **travel time** and **fuel consumption** while **avoiding traffic congestion**. This enhances the user experience by
ensuring quicker and more efficient rides.

Given Input Data:

GPS data, traffic reports, historical route performance, and road conditions.

### Choosing the target

Depends on the primary goal of the optimization model

1. Travel Time: in minutes between pickup and destination
    - This is the most common choice for route optimization
2. Fuel Consumption: Estimated fuel consumption in litres or gallons
    - This might not be possible without vehicle type
3. Trip Cost or Fare: Fare charged or total cost

Given the use case, 1. and 2. are most likely. It is also possible to combine these with composite scores. Some examples:
- I need to do more research for this
